In [9]:
import requests, json
import pandas as pd
from datetime import datetime as dt, timedelta, date
import time
import pytz
from base64 import b64encode, b64decode
from trading_ig_config import config
from Crypto.Cipher import PKCS1_v1_5
from Crypto.PublicKey import RSA

class Config:
    def __init__(self, account_type):
        if account_type == 'LIVE':
            self.acc_type = 'LIVE'
            self.endpoint = 'https://api.ig.com/gateway/deal'
        else:
            self.acc_type = 'DEMO'
            self.endpoint = 'https://demo-api.ig.com/gateway/deal'
            
class IG:
    def __init__(self, key, username, password, account, acc_type='DEMO'):
        self.api_key = key
        self.username = username
        self.password = password
        self.account_type = acc_type
        self.account_num = ''
        self.CST = ''
        self.X_SECURITY_TOKEN = ''
        self.lightstreamerEndpoint = ''
        self.s = requests.Session()
        self.config = Config(self.account_type)
        self.ls_client = None
        self.version = '1.0.6'
        
    def getVersion(self):
        return self.version

    def encrypted_password(self, key, timestamp, password):
        rsakey = RSA.importKey(b64decode(key))
        string = password + "|" + str(int(timestamp))
        message = b64encode(string.encode())
        return b64encode(PKCS1_v1_5.new(rsakey).encrypt(message)).decode()

    def login(self):
        response = self.s.get(self.config.endpoint+'/session/encryptionKey', headers={'X-IG-API-KEY': self.api_key, 'Content-Type': 'application/json; charset=UTF-8', 'Accept': 'application/json; charset=UTF-8'})
        data = response.json()
        encryption_key = data['encryptionKey']
        timestamp = data['timeStamp']
        encrypted_password = self.encrypted_password(encryption_key,timestamp,self.password)
        response = self.s.post(self.config.endpoint+'/session', headers={'X-IG-API-KEY': self.api_key, 'Content-Type': 'application/json; charset=UTF-8', 'Accept': 'application/json; charset=UTF-8', 'VERSION': '2'}, data=json.dumps({"identifier":self.username, "password":encrypted_password,"encryptedPassword":True}))
        self.CST = response.headers['CST']
        print(self.CST)
        self.X_SECURITY_TOKEN = response.headers['X-SECURITY-TOKEN']
        print(self.X_SECURITY_TOKEN)
    

In [10]:
ig=IG(config.api_key, config.username, config.password, config.acc_type)
ig.login()


'f0e2605109cbf6ee60fe13739cae92498965061502813699535eaa4ccc74b4CD01111'